# Introduction

### Headers
- Benchmark Name
- Total Depenedencies
- Dependencies Ratio
- Realizable (If one of the tools claims for realizability)
- (Tool name) Duration
- (Tool name) Status (ERROR | COMPLETED | TIMEOUT)

### Tools
 - Deps Synthesis (Synthesis 2 strategies: dependents, indepenents)
 - Spot Hijack (Don't find deps + don't eject)
 - Indep Synthesis (Synhesis a strategy only for independents, i.e., remove dependents from specification)
 - Spot Split-Determinstic (SD)
 - Spot LAR
 - Spot PS
 - Strix

### Sanity Check
- Realizability is similar in all tools

In [1]:
import pandas as pd
from matplotlib import pyplot as plt
from glob import glob
import os.path
from pathlib import Path
import json
import re

In [2]:
def seconds_to_ms(secs):
    return secs * 1000

TIMEOUT_MS = seconds_to_ms(60*39) # 39 minutes

# Loading Results

## Deps Synthesis

In [3]:
# TODO: all paths extract to global const file
deps_synt_hoa_files = glob("../assets/aiger-synthesis/bfss-with-deps/*.hoa")

def deps_synt_analysis(benchmark_name):
    hoa_file_path = "../assets/aiger-synthesis/bfss-with-deps/{}.hoa".format(benchmark_name)
    json_file_path = "../assets/aiger-synthesis/bfss-with-deps/{}.json".format(benchmark_name)
    hoa_file = Path(hoa_file_path).read_text()
    
    # If JSON file doesn't exist, it we had error in synthesis
    if not os.path.exists(json_file_path):
        status = 'error'
        error = ','.join(re.findall('Runtime error:(.*)', hoa_file))
        
        return {
            'name': benchmark_name,
            'status': 'error',
            'error': error
        }
    
    measures = json.loads(Path(json_file_path).read_text())
    
    duration = measures['total_time']
    status = 'completed' if measures['is_completed'] == True  else 'timeout'
    if status != 'completed':
        return {
            'name': benchmark_name,
            'duration': duration,
            'status': status,
        }
    
    total_dependents = sum((1 for v in measures['tested_variables'] if v['is_dependent']))
    total_outputs = len(measures['output_vars'])
    dependents_ratio = total_dependents / total_outputs
    realizablility = measures['synthesis']['independent_strategy']['realizability']
    
    return {
        'name': benchmark_name,
        'duration': duration,
        'status': status,
        'dependents_ratio': dependents_ratio,
        'total_dependents': total_dependents,
        'realizablility': realizablility
    }

deps_synt_df = pd.DataFrame([
    deps_synt_analysis(Path(benchmark_path).stem)
    for benchmark_path in deps_synt_hoa_files
])


deps_synt_df

,name,duration,status,dependents_ratio,total_dependents,realizablility,error
0,TwoCountersInRangeA4,210.0,completed,0.375000,3.0,UNREALIZABLE,NaN
1,ltl2dba05,6.0,completed,0.000000,0.0,REALIZABLE,NaN
2,ltl2dba11,2.0,completed,0.000000,0.0,REALIZABLE,NaN
3,TwoCountersDisButA6,12926.0,completed,0.307692,4.0,UNREALIZABLE,NaN
4,TorcsSteeringSimple,5.0,completed,0.333333,1.0,REALIZABLE,NaN
...,...,...,...,...,...,...,...
202,OneCounterGuiA1,22.0,completed,0.333333,3.0,UNREALIZABLE,NaN
203,EscalatorSmart,8.0,completed,0.333333,1.0,REALIZABLE,NaN
204,OneCounterGuiA0,41.0,completed,0.333333,3.0,UNREALIZABLE,NaN
205,ltl2dba20,9.0,completed,0.000000,0.0,REALIZABLE,NaN


In [4]:
deps_synt_df[deps_synt_df['status'] != 'completed']

,name,duration,status,dependents_ratio,total_dependents,realizablility,error
63,08,NaN,error,NaN,NaN,NaN,Too many acceptance sets used. The limit is 32.
65,09,NaN,error,NaN,NaN,NaN,Too many acceptance sets used. The limit is 32.
75,KitchenTimerV7,2399969.0,timeout,NaN,NaN,NaN,NaN
79,KitchenTimerV6,2400021.0,timeout,NaN,NaN,NaN,NaN
91,lilydemo18,NaN,error,NaN,NaN,NaN,Too many acceptance sets used. The limit is 32.
141,13,NaN,error,NaN,NaN,NaN,Too many acceptance sets used. The limit is 32.
142,07,NaN,error,NaN,NaN,NaN,Too many acceptance sets used. The limit is 32.
144,06,NaN,error,NaN,NaN,NaN,Too many acceptance sets used. The limit is 32.
145,12,NaN,error,NaN,NaN,NaN,Too many acceptance sets used. The limit is 32.
154,ltl2dpa22,2399977.0,timeout,NaN,NaN,NaN,NaN


## SPOT SD

In [9]:
# TODO: all paths extract to global const file
spot_sd_hoa_files = glob("../assets/aiger-synthesis/spot-sd/*.hoa")

def spot_sd_analysis(benchmark_name):
    hoa_file_path = "../assets/aiger-synthesis/spot-sd/{}.hoa".format(benchmark_name)
    hoa_file = Path(hoa_file_path).read_text()
    
    duration_str = re.findall('real\s+(.*)m(.*)s', hoa_file)
    if len(duration_str) != 1:
        raise Exception(f'Duration not found for benchmark {benchmark_name}, duration = ', duration)

    duration_minutes = float(duration_str[0][0])
    duration_seconds = float(duration_str[0][1])
    duration = seconds_to_ms(duration_seconds + (60*duration_minutes))
    
    is_completed = ('UNREALIZABLE' in hoa_file) or ('REALIZABLE' in hoa_file)
    if not is_completed:
        status = 'timeout' if duration >= TIMEOUT_MS else 'error'
        
        if status == 'error':
            error = ', '.join(re.findall('ltlsynt:(.*)', hoa_file))
        else:
            error = None
        
        return {
            'name': benchmark_name,
            'status': status,
            'error': error,
            'duration': duration
        }
    
    realizablility = 'unrealizable' if 'UNREALIZABLE' in hoa_file else 'realizable'
    return {
        'name': benchmark_name,
        'status': 'completed',
        'duration': duration,
        'realizablility': realizablility
    }

spot_sd_df = pd.DataFrame([
    spot_sd_analysis(Path(benchmark_path).stem)
    for benchmark_path in spot_sd_hoa_files
])

spot_sd_df

,name,status,duration,realizablility,error
0,TwoCountersInRangeA4,completed,2605.0,unrealizable,NaN
1,ltl2dba05,completed,35.0,realizable,NaN
2,ltl2dba11,completed,36.0,realizable,NaN
3,TwoCountersDisButA6,completed,2000596.0,unrealizable,NaN
4,TorcsSteeringSimple,completed,41.0,realizable,NaN
...,...,...,...,...,...
202,OneCounterGuiA1,completed,191.0,unrealizable,NaN
203,EscalatorSmart,completed,38.0,realizable,NaN
204,OneCounterGuiA0,completed,68.0,unrealizable,NaN
205,ltl2dba20,completed,37.0,realizable,NaN


In [10]:
spot_sd_df[spot_sd_df['status'] != 'completed']

,name,status,duration,realizablility,error
13,TwoCountersDisButAC,timeout,2400211.0,NaN,None
63,08,error,179.0,NaN,Too many acceptance sets used. The limit is 32.
65,09,error,157.0,NaN,Too many acceptance sets used. The limit is 32.
97,SensorSelector,timeout,2400285.0,NaN,None
115,LedMatrix,timeout,2400176.0,NaN,None
129,02,error,100.0,NaN,alternate_players(): Odd cycle detected.
131,03,error,2416.0,NaN,alternate_players(): Odd cycle detected.
141,13,error,185.0,NaN,Too many acceptance sets used. The limit is 32.
142,07,error,88.0,NaN,Too many acceptance sets used. The limit is 32.
144,06,error,170.0,NaN,Too many acceptance sets used. The limit is 32.


## Spot Hijacked

,name,status,duration,realizablility,error
0,TwoCountersInRangeA4,completed,2605.0,unrealizable,NaN
1,ltl2dba05,completed,35.0,realizable,NaN
2,ltl2dba11,completed,36.0,realizable,NaN
3,TwoCountersDisButA6,completed,2000596.0,unrealizable,NaN
4,TorcsSteeringSimple,completed,41.0,realizable,NaN
...,...,...,...,...,...
202,OneCounterGuiA1,completed,191.0,unrealizable,NaN
203,EscalatorSmart,completed,38.0,realizable,NaN
204,OneCounterGuiA0,completed,68.0,unrealizable,NaN
205,ltl2dba20,completed,37.0,realizable,NaN


# Comparing Results

## Full Specification vs Independent Specification 
How much synthesis independent only improved the process?

I.e., how much is synthesis the specificaiton with independent variables is better than synthesis the full specficiation?